In [3]:
from pathlib import Path
import numpy as np
import torch
import pickle

In [4]:
openmask3d_matches_path = "/home/kumaraditya/openlex3d/query_experiments_perf_debug_v2/viz/replica/room0/openmask3d_nms_0.5_duplicated_per_mask_0.002_top_k_0.5_20"
cg_matches_path = "/home/kumaraditya/openlex3d/query_experiments_perf_debug_v2/viz/replica/room0/concept-graphs_global_0.01_top_k_0.5_20"
openmask3d_ranks_path = "/home/kumaraditya/openlex3d/query_experiments_perf_debug_v2/rank_metric/replica/openmask3d_nms_0.5_duplicated_per_mask_0.002_20_0.25"
cg_ranks_path = "/home/kumaraditya/openlex3d/query_experiments_perf_debug_v2/rank_metric/replica/concept-graphs_global_0.01_20_0.25"

In [5]:
openmask3d_matches_path = Path(openmask3d_matches_path)
cg_matches_path = Path(cg_matches_path)
openmask3d_ranks_path = Path(openmask3d_ranks_path)
cg_ranks_path = Path(cg_ranks_path)

In [6]:
openmask3d_matches = pickle.load(open(openmask3d_matches_path / "matches.pkl", "rb"))
cg_matches = pickle.load(open(cg_matches_path / "matches.pkl", "rb"))
openmask3d_ranks = pickle.load(open(openmask3d_ranks_path / "query_ranks.pkl", "rb"))
cg_ranks = pickle.load(open(cg_ranks_path / "query_ranks.pkl", "rb"))

In [12]:
scene = "room0"

In [13]:
def get_pred_ids(matches, target_query):
    unique_masks = set()
    mask_indices_list = []  # Store all arrays
    for pred in matches[scene]["pred"]["object"]:
        query = pred["query_id"].split("_")[1]
        if query != target_query:
            continue
        print(f'{pred["pred_id"]}:{pred["rank"]}')
        mask_array = tuple(
            sorted(pred["mask_indices"])
        )  # Convert to a tuple to make it hashable
        mask_indices_list.append(mask_array)
        unique_masks.add(mask_array)  # Add to set for uniqueness check
    # Count unique arrays
    num_unique_masks = len(unique_masks)
    # Print results
    print(f"Total mask_indices arrays: {len(mask_indices_list)}")
    print(f"Unique mask_indices arrays: {num_unique_masks}")

In [14]:
get_pred_ids(openmask3d_matches, "vase")

15:1
41:2
69:3
33:4
62:5
19:6
60:7
65:8
0:9
8:10
14:11
5:12
44:13
37:14
58:15
34:16
9:17
49:18
68:19
35:20
Total mask_indices arrays: 20
Unique mask_indices arrays: 20


In [24]:
# print query_ids with only -1 ranks
openmask3d_bad_queries = []
for scene_id, query_data in openmask3d_ranks.items():
    if scene_id != scene:
        continue
    for query_id, ranks in query_data.items():
        if all(r == -1 for r in ranks):
            query_text = query_id.split("_")[1]
            openmask3d_bad_queries.append(query_text)

concept_graph_bad_queries = []
for scene_id, query_data in cg_ranks.items():
    if scene_id != scene:
        continue
    for query_id, ranks in query_data.items():
        if all(r == -1 for r in ranks):
            query_text = query_id.split("_")[1]
            concept_graph_bad_queries.append(query_text)

# convert the list of strings to a set
openmask3d_bad_queries = set(openmask3d_bad_queries)
concept_graph_bad_queries = set(concept_graph_bad_queries)

# print the queries that are bad for only openmask3d
only_openmask_bad_queries = list(openmask3d_bad_queries - concept_graph_bad_queries)
print(len(only_openmask_bad_queries))

only_cg_bad_queries = list(concept_graph_bad_queries - openmask3d_bad_queries)
print(len(only_cg_bad_queries))

print(len(openmask3d_bad_queries))
print(len(concept_graph_bad_queries))

print(only_openmask_bad_queries[:20])

39
16
107
84
['shades', 'carpet floor', 'grate', 'throw blanket', 'switch', 'grille', 'register vent', 'air vent', 'fluorescent lighting', 'grating', 'art', 'picture', 'tube light', 'flat panel light', 'fluorescent lamp', 'blanket', 'vent', 'panel lighting', 'throw', 'carpeted floor']


In [10]:
# get number of -1 ranks for all the data in ranks
openmask3d_total_bad = 0
for scene_id, query_data in openmask3d_ranks.items():
    for query_id, ranks in query_data.items():
        openmask3d_total_bad += sum(r == -1 for r in ranks)

cg_total_bad = 0
for scene_id, query_data in cg_ranks.items():
    for query_id, ranks in query_data.items():
        cg_total_bad += sum(r == -1 for r in ranks)

print(openmask3d_total_bad)
print(cg_total_bad)

1710
1408


### Bare Check

In [11]:
bare_matches_path = "/home/kumaraditya/openlex3d/query_experiments_overnight/viz/replica/room0/bare_global_0.01_top_k_0.5_20"
bare_ranks_path = "/home/kumaraditya/openlex3d/query_experiments_overnight/rank_metric/replica/bare_global_0.01_20_0.25"

bare_matches_path = Path(bare_matches_path)
bare_ranks_path = Path(bare_ranks_path)

bare_matches = pickle.load(open(bare_matches_path / "matches.pkl", "rb"))
bare_ranks = pickle.load(open(bare_ranks_path / "query_ranks.pkl", "rb"))

In [25]:
bare_bad_queries = []
for scene_id, query_data in bare_ranks.items():
    if scene_id != scene:
        continue
    for query_id, ranks in query_data.items():
        if query_id.split("_")[0] == "level1":
            continue
        if all(r == -1 for r in ranks):
            query_text = query_id.split("_")[1]
            bare_bad_queries.append(query_text)

bare_bad_queries = set(bare_bad_queries)

only_bare_bad_queries = list(
    bare_bad_queries - concept_graph_bad_queries - openmask3d_bad_queries
)
print(len(only_bare_bad_queries))

33


In [26]:
print(only_bare_bad_queries[:20])

['painting', 'photograph', 'bud vase', 'armchair', 'printed canvas', 'window pane', 'seat', 'mirror', 'wall mounted mirror', 'door', 'glass vase', 'decorative branches', 'decorative mirror', 'canvas', 'bouquet', 'divan', 'ceiling light', 'hardcover', 'wall mirror', 'wall art']


## OpenMask3D Masks Check

In [ ]:
openmask3d_masks_path = "/home/kumaraditya/openmask3d/openmask3d/output/replica/room0/2025-01-30-15-19-26-final-eval-10/room0_mesh_masks.pt"
openmask3d_masks = np.asarray(torch.load(openmask3d_masks_path, weights_only=False)).T
openmask3d_masks.shape